# Reddit Depression Final Project
Link to the paper: https://dl.acm.org/doi/pdf/10.1145/3578503.3583621

Read through the paper fully before starting the assignment!

In [39]:
pip install happiestfuntokenizing

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier

from google.colab import drive
drive.mount('/content/drive')

FILEPATH = 'drive/MyDrive/NLP/Final'
loaded_data = None

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [41]:
import pickle
def load():
  """Load pickles"""
  with open(FILEPATH + '/student.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
  return loaded_data

load().shape

(1958158, 5)

In [42]:
# List of depression subreddits in the paper
depression_subreddits = ["Anger",
    "anhedonia", "DeadBedrooms",
    "Anxiety", "AnxietyDepression", "HealthAnxiety", "PanicAttack",
    "DecisionMaking", "shouldi",
    "bingeeating", "BingeEatingDisorder", "EatingDisorders", "eating_disorders", "EDAnonymous",
    "chronicfatigue", "Fatigue",
    "ForeverAlone", "lonely",
    "cry", "grief", "sad", "Sadness",
    "AvPD", "SelfHate", "selfhelp", "socialanxiety", "whatsbotheringyou",
    "insomnia", "sleep",
    "cfs", "ChronicPain", "Constipation", "EssentialTremor", "headaches", "ibs", "tinnitus",
    "AdultSelfHarm", "selfharm", "SuicideWatch",
    "Guilt", "Pessimism", "selfhelp", "whatsbotheringyou"
]

symptoms_2_subreddit = {"anger":["Anger"],
                          "anhedonia":["anhedonia", "DeadBedrooms"],
                          "anxiety":['Anxiety', 'AnxietyDepression', 'HealthAnxiety', 'PanicAttack'],
                          "concentration deficit" : ['DecisionMaking', "shouldi"],
                          "disordered eating" :["bingeeating", "BingeEatingDisorder", "EatingDisorders", "eating_disorders", "EDAnonymous"],
                         "fatigue":["chronicfatigue", "Fatigue"],
                         "loneliness":["ForeverAlone", "lonely"],
                          "sad mood":["cry", "grief", "sad", "Sadness"],
                          "self-loathing":["AvPD", "SelfHate", "selfhelp", "socialanxiety", "whatsbotheringyou"],
                          "sleep problem":["insomnia", "sleep"],
                          "somatic complaint":["cfs", "ChronicPain", "Constipation", "EssentialTremor", "headaches", "ibs", "tinnitus"],
                         "suicidal thoughts and attempts":[ "AdultSelfHarm", "selfharm", "SuicideWatch"],
                          "worthlessness":["Guilt", "Pessimism", "selfhelp", "whatsbotheringyou"]}

all_symptoms = ["anger", "anhedonia",
                          "anxiety",
                          "concentration deficit",
                          "disordered eating",
                         "fatigue",
                         "loneliness",
                          "sad mood",
                          "self-loathing",
                          "sleep problem",
                          "somatic complaint",
                         "suicidal thoughts and attempts",
                          "worthlessness"]


In [43]:
from datetime import datetime, timedelta

def dataset_generation():
  """Build control and symptom datasets"""

  # anger, anhedonia, anxiety, concentration_deficit, disordered_eating, fatigue, loneliness, sad_mood, self_loathing, sleep_problem, somatic_complaint, suicidal, worthlessness = [[] for _ in range(13)]

  def symptom_type_condition(sub):
    if sub in depression_subreddits:
      if sub in symptoms_2_subreddit["anger"]:
        return "anger"
      if sub in symptoms_2_subreddit["anhedonia"]:
        return "anhedonia"
      if sub in symptoms_2_subreddit["anxiety"]:
        return "anxiety"
      if sub in symptoms_2_subreddit["concentration deficit"]:
        return "concentration deficit"
      if sub in symptoms_2_subreddit["disordered eating"]:
        return "disordered eating"
      if sub in symptoms_2_subreddit["fatigue"]:
        return "fatigue"
      if sub in symptoms_2_subreddit["loneliness"]:
        return "loneliness"
      if sub in symptoms_2_subreddit["sad mood"]:
        return "sad mood"
      if sub in symptoms_2_subreddit["self-loathing"]:
        return "self-loathing"
      if sub in symptoms_2_subreddit["sleep problem"]:
        return "sleep problem"
      if sub in symptoms_2_subreddit["somatic complaint"]:
        return "somatic complaint"
      if sub in symptoms_2_subreddit["suicidal thoughts and attempts"]:
        return "suicidal thoughts and attempts"
      if sub in symptoms_2_subreddit["worthlessness"]:
        return "worthlessness"
    else:
        return "control"

  #cleaning data
  loaded_data = load()
  #remove deleted
  loaded_data = loaded_data[loaded_data["author"]!="[deleted]"]
  symptom_data = loaded_data[loaded_data['subreddit'].isin(depression_subreddits)]
  control_data = loaded_data[~(loaded_data['subreddit'].isin(depression_subreddits))]

  control_text = []
  symptom_text = []
  auth_2_oldest_dep_post = {}

  #SYMPTOM
  for index, row in symptom_data.iterrows():
      symptom_text.append(row['text'])
      #keep track of oldest depression post by this author
      if row['author'] in auth_2_oldest_dep_post:
        auth_2_oldest_dep_post[row['author']] = min(auth_2_oldest_dep_post[row['author']], int(row['created_utc']))
      else:
        auth_2_oldest_dep_post[row['author']] = int(row['created_utc']) #first time seeing a depression post by this author
  symptom_data['symptom_type'] = symptom_data['subreddit'].apply(symptom_type_condition)

  #CONTROL
  for index, row in control_data.iterrows():
    #author with a depression post with current post not being dep post
    if row['author'] in auth_2_oldest_dep_post:
      if 180 <= (auth_2_oldest_dep_post[row['author']] - row['created_utc'])/86400 :
        #dep post needs to be at least 180 days older than non-dep
        control_text.append(row['text'])
  control_data = control_data[control_data['text'].isin(control_text)]
  control_data['symptom_type'] = control_data['subreddit'].apply(symptom_type_condition)

  #controltxt is list of txts, symptom is list of txt, control data and symptom data have a symtom-type column
  return control_text, symptom_text, control_data, symptom_data

In [74]:
c, s, c_data, s_data = dataset_generation()
lda_all_df = pd.concat([c_data, s_data], axis=0, ignore_index=True) #global datafram
# remove empty arrays
lda_all_df = lda_all_df[lda_all_df['text'].apply(lambda x: len(x) > 0)]

<ipython-input-43-08e33b03a5a7>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  symptom_data['symptom_type'] = symptom_data['subreddit'].apply(symptom_type_condition)


In [75]:
from happiestfuntokenizing.happiestfuntokenizing import Tokenizer
def tokenize(control_txts, sym_txts):
  """Tokenize"""
  #words across datasets
  tokenizer = Tokenizer()
  control_tokens = []
  symptom_tokens = []
  for sentence in control_txts:
    curr_tokens = tokenizer.tokenize(sentence)
    control_tokens.append(curr_tokens) #list of list of sentence-wise tokens
  for sentence in sym_txts:
      curr_tokens = tokenizer.tokenize(sentence)
      symptom_tokens.append(curr_tokens) #list of list of sentence wise tokens, clustered symptom-wise but not further nesting
  # print(control_tokens[0])
  lda_all_df['tokenized_text'] = lda_all_df['text'].apply(tokenizer.tokenize)
  return control_tokens, symptom_tokens


In [77]:
print(lda_all_df)

                                                     text  \
0                                           2meirl4meirl    
1                Man, I do love me some Bandicoot crash.    
2                                                 me irl    
3       How good is this PC for my 700-750$ budget? Wa...   
4                                              anime_irl    
...                                                   ...   
107030  I think my DB has made me crazy. This morning ...   
107031                    An unsuccessful story [removed]   
107032  Weed cured my severe insomnia. The past few mo...   
107033        How many refusers are on this sub and why?    
107034  My act of self care: this weighted blanket. It...   

                     author        subreddit  created_utc     date  \
0          MyUserIdWasTaken     2meirl4meirl   1506639061  2017-09   
1       BuddermanTheAmazing  crappyoffbrands   1499236239  2017-07   
2            Mank_____Demes           me_irl   1515641342

In [76]:
c_tokens, s_tokens = tokenize(c, s)

In [79]:
from gensim.corpora import Dictionary
from collections import Counter
def stop_words(control_tokens, symptom_tokens):
  """Find top 100 words from Reddit dataset to use as stop words"""
  # remove stop across full vocab
  all_tokens = control_tokens+symptom_tokens

  word_counter = Counter([word for sublist in all_tokens for word in sublist])
  top_100 = [word for word, freq in word_counter.most_common(100)]

  id2token = Dictionary(all_tokens)
  stop_ids = [id2token.token2id[word] for word in top_100 if word in id2token.token2id]
  id2token.filter_tokens(bad_ids=stop_ids) #removing stopwords from id to token mapping


  lda_all_df['stop_free_text'] = lda_all_df['tokenized_text'].apply(lambda x: [word for word in x if word not in top_100])
  doc2bow_matrix = [id2token.doc2bow(sentence) for sentence in all_tokens] #makes each document/sentence a
  #bag of words (using ids instead of actual words)
  return id2token, doc2bow_matrix

In [80]:
print(lda_all_df)

                                                     text  \
0                                           2meirl4meirl    
1                Man, I do love me some Bandicoot crash.    
2                                                 me irl    
3       How good is this PC for my 700-750$ budget? Wa...   
4                                              anime_irl    
...                                                   ...   
107030  I think my DB has made me crazy. This morning ...   
107031                    An unsuccessful story [removed]   
107032  Weed cured my severe insomnia. The past few mo...   
107033        How many refusers are on this sub and why?    
107034  My act of self care: this weighted blanket. It...   

                     author        subreddit  created_utc     date  \
0          MyUserIdWasTaken     2meirl4meirl   1506639061  2017-09   
1       BuddermanTheAmazing  crappyoffbrands   1499236239  2017-07   
2            Mank_____Demes           me_irl   1515641342

In [81]:
id2token, doc2bow_matrix = stop_words(c_tokens, s_tokens)

## Reddit Topics with LDA

 - Don't use MALLET (as the paper does), use some other LDA implementation.

In [52]:
from gensim.models.ldamulticore import LdaMulticore
# TODO: Your LDA code!
# document with each word in vocab as yes no
#train on everything
#the corpus is where each row is a document represented as the words in it (each row is a bow vector)
#after training we want to find what topic each bow_vector becomes
#overe everything
lda_model = LdaMulticore(corpus=doc2bow_matrix,
                         id2word=id2token,
                         num_topics=200,
                         random_state=100) #random state improves reproducibility

#save trained model
with open(FILEPATH + "lda_model.pkl", "wb") as f:
    pickle.dump(lda_model, f)

#after we train lda model on corpus which is each doument as a bow vector, where words are
#represented by ids as in id2word, we then want to get the probability distribution of topics
#in each document
#when we do get_documt_topics, we get tuples representing the id of the topic and how probable that topic
#is in the document

In [82]:
def lda_subset_gen(ith_symptom):
  # subset embeddings by control + symptom
  keep_row_condition = lda_all_df['symptom_type'].apply(lambda x: x=="control" or x == ith_symptom)
  subset = lda_all_df[keep_row_condition]
  subset = subset.reset_index(drop=True) #start from 0 again

  subset['labels'] = subset['symptom_type'].apply(lambda x: 1 if x == ith_symptom else 0)
  #get corpus for each symptom
  sym_doc2bow_matrix = [id2token.doc2bow(txt) for txt in subset['stop_free_text']] #makes each document/sentence a
  labels_list = np.array(list(subset['labels']))

  return sym_doc2bow_matrix, labels_list

In [84]:
sym2corpus, ldasym2labels = {}, {} #collection of each symptom's symptom post+control embeddings,
#collection of each symptom's post labels
for symptom in all_symptoms:
  sym_doc2bow_matrix, labels_list = lda_subset_gen(symptom)
  sym2corpus[symptom] = sym_doc2bow_matrix
  ldasym2labels[symptom] = labels_list

0                                           eirl4meirl]
1                            [man, love, bandicoot, crash]
2                                                    [irl]
3        [good, pc, 700-750, $, budget, gaming, high, u...
4                                              [anime_irl]
                               ...                        
13415    [stop, before, gets, hand, hopefully, y'all, a...
13416                     [hypocrisy, real, [, removed, ]]
13417    [hate, after, losing, control, first, posting,...
13418    [m, particularly, angry, person, today, after,...
13419    [sometimes, literally, neurology, shifting, po...
Name: stop_free_text, Length: 13420, dtype: object
0                                           eirl4meirl]
1                            [man, love, bandicoot, crash]
2                                                    [irl]
3        [good, pc, 700-750, $, budget, gaming, high, u...
4                                              [anime_irl]
           

In [86]:
sym2topicembedding = {}
for key in sym2corpus:
  doc_embeddings = []
  print("sym2corpus", sym2corpus)
  for doc in sym2corpus[key]: #each row is a doc represented by a bow vector

    list_of_tuples = lda_model.get_document_topics(doc, minimum_probability=0)
    #each tuple is a topic id with the prob that topic is in this doc
    doc_topic_probs = [topic_prob for topic_id, topic_prob in list_of_tuples]
    doc_embeddings.append(doc_topic_probs)
  sym2topicembedding[key] = doc_embeddings

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



sym2corpus 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



sym2corpus 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



sym2corpus 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



sym2corpus 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## RoBERTa Embeddings

In [14]:
# set up cuda
import torch
torch.cuda.empty_cache()
device = torch.device("cuda")

In [15]:
# TODO: Your RoBERTa code!
import math
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaModel
tqdm.pandas()

all_df = pd.concat([c_data, s_data], axis=0, ignore_index=True) #global datafram
# remove empty arrays
all_df = all_df[all_df['text'].apply(lambda x: len(x) > 0)]

#using pretrained models
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base', output_hidden_states=True)
model.to(device)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [16]:
def tokenize_posts(text):
    input_posts = tokenizer(text, return_tensors = "pt", padding= True, truncation = True, is_split_into_words = True, max_length = 512)
    input_posts = {key: value.to('cuda') for key, value in input_posts.items()}
    return input_posts

all_df['roberta_input_posts'] = all_df['text'].progress_apply(tokenize_posts)

100%|██████████| 107035/107035 [05:03<00:00, 352.16it/s]


                                                     text  \
0                                           2meirl4meirl    
1                Man, I do love me some Bandicoot crash.    
2                                                 me irl    
3       How good is this PC for my 700-750$ budget? Wa...   
4                                              anime_irl    
...                                                   ...   
107030  I think my DB has made me crazy. This morning ...   
107031                    An unsuccessful story [removed]   
107032  Weed cured my severe insomnia. The past few mo...   
107033        How many refusers are on this sub and why?    
107034  My act of self care: this weighted blanket. It...   

                     author        subreddit  created_utc     date  \
0          MyUserIdWasTaken     2meirl4meirl   1506639061  2017-09   
1       BuddermanTheAmazing  crappyoffbrands   1499236239  2017-07   
2            Mank_____Demes           me_irl   1515641342

In [17]:
from torch import autocast
def forward_pass(input):
  with autocast(device_type='cuda'):
    output = model(**input)
  avg = torch.mean(output.hidden_states[9], dim=1)
  return avg.cpu().detach().numpy()

In [18]:
with torch.no_grad():
  all_df['output_avg_post_embeddings'] = all_df['roberta_input_posts'].progress_apply(forward_pass)

100%|██████████| 107035/107035 [32:19<00:00, 55.18it/s]


In [20]:
with open(FILEPATH+ '/trained_roberta.pkl', 'wb') as file:
      pickle.dump(all_df, file)

In [32]:
def roberta_subset_gen(ith_symptom):
  # subset embeddings by control + symptom
  keep_row_condition = all_df['symptom_type'].apply(lambda x: x=="control" or x == ith_symptom)
  subset = all_df[keep_row_condition]
  subset = subset.reset_index(drop=True) #start from 0 again

  subset['labels'] = subset['symptom_type'].apply(lambda x: 1 if x == ith_symptom else 0)
  #get post embeddings
  post_embs_list = torch.tensor(subset['output_avg_post_embeddings']).squeeze(dim=1)
  labels_list = np.array(list(subset['labels']))

  return post_embs_list, labels_list

In [31]:
sym2embeddings, sym2labels = {}, {} #collection of each symptom's symptom post+control embeddings,
#collection of each symptom's post labels
for symptom in all_symptoms:
  post_embs_list, labels_list = roberta_subset_gen(symptom)
  sym2embeddings[symptom] = post_embs_list
  sym2labels[symptom] = labels_list

                                                    text               author  \
0                                          2meirl4meirl      MyUserIdWasTaken   
1               Man, I do love me some Bandicoot crash.   BuddermanTheAmazing   
2                                                me irl        Mank_____Demes   
3      How good is this PC for my 700-750$ budget? Wa...  WildernessExploring   
4                                             anime_irl         Cake_Lancelot   
...                                                  ...                  ...   
13415  How To Stop This Before it Gets Out Of Hand? H...       turtlemachines   
13416         Hypocrisy has never been so real [removed]              C4Sidhu   
13417  I hate myself after losing control First time ...  itjustkeepsbuilding   
13418  I’m not a particularly angry person, but today...              Matt-32   
13419  Sometimes I can literally feel my neurology sh...      umbilicasillica   

             subreddit  cre

## Main

In [34]:
def main(X, y):
  """
  Here's the basic structure of the main block! It should run
  5-fold cross validation with random forest to evaluate your RoBERTa and LDA
  performance.
  """
  rf_classifier = RandomForestClassifier()
  cv = KFold(n_splits=5, shuffle=True)
  results = cross_validate(rf_classifier, X=X, y=y, cv=cv, scoring='roc_auc', return_train_score=True)

  # TODO: Print your training and testing scores!
  return (results)

lda_auc_scores = {} #symptom to auc
for symptom in all_symptoms:
  dict_output =  main(sym2topicembedding[symptom], ldasym2labels[symptom])
  lda_auc_scores[symptom] = dict_output['test_score'].mean()
lda_scores_df = pd.DataFrame.from_dict(lda_auc_scores, orient='index', columns=['Score'])
lda_scores_df.reset_index(inplace=True)
lda_scores_df.columns = ['Symptom Type', 'Score']
print("lda results:", lda_auc_scores)
print("lda dataframe:", lda_scores_df.dropna())


roberta_scores = {} #symptom to auc
for symptom in all_symptoms:
  dict_output =  main(sym2embeddings[symptom], sym2labels[symptom])
  roberta_scores[symptom] = dict_output['test_score'].mean()
roberta_scores_df = pd.DataFrame.from_dict(roberta_scores, orient='index', columns=['Score'])
roberta_scores_df.reset_index(inplace=True)
roberta_scores_df.columns = ['Symptom Type', 'Score']
print("roberta results:", roberta_scores)
print("roberta dataframe:", roberta_scores_df.dropna())



In [87]:
lda_auc_scores = {} #symptom to auc
for symptom in all_symptoms:
  dict_output =  main(sym2topicembedding[symptom], ldasym2labels[symptom])
  lda_auc_scores[symptom] = dict_output['test_score'].mean()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 399, in _score
    return self._sign * self._score_func(y, y_pred, **self

In [97]:
lda_df_scores = pd.DataFrame.from_dict(lda_auc_scores, orient='index', columns=['Score'])

# Resetting index (optional)
lda_df_scores.reset_index(inplace=True)
lda_df_scores.columns = ['Symptom', 'Score']
print("lda dataframe:\n", lda_df_scores.dropna())

lda dataframe:
                            Symptom     Score
0                            anger  0.976413
1                        anhedonia  0.985150
2                          anxiety  0.971655
4                disordered eating  0.980746
6                       loneliness  0.957989
7                         sad mood  0.944486
8                    self-loathing  0.957548
9                    sleep problem  0.992119
10               somatic complaint  0.971224
11  suicidal thoughts and attempts  0.978596
12                   worthlessness  0.693558


In [35]:
roberta_scores = {} #symptom to auc
for symptom in all_symptoms:
  dict_output =  main(sym2embeddings[symptom], sym2labels[symptom])
  roberta_scores[symptom] = dict_output['test_score'].mean()

SYMPTOM: anger AUC: 0.985261610290386
SYMPTOM: anhedonia AUC: 0.9873863301752135
SYMPTOM: anxiety AUC: 0.9859090661422785


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 399, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py", line 572, in roc_auc_score
    return _average_binary_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_base.py", line 75, in _average_binary_score
    return binary_metric(y_true, y_score, sample_weight=sample_weight)

SYMPTOM: concentration deficit AUC: nan
SYMPTOM: disordered eating AUC: 0.9881735086818335


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 399, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py", line 572, in roc_auc_score
    return _average_binary_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_base.py", line 75, in _average_binary_score
    return binary_metric(y_true, y_score, sample_weight=sample_weight)

SYMPTOM: fatigue AUC: nan
SYMPTOM: loneliness AUC: 0.9735799399067208
SYMPTOM: sad mood AUC: 0.9798452067209233
SYMPTOM: self-loathing AUC: 0.9788697683254043
SYMPTOM: sleep problem AUC: 0.9912748997067256
SYMPTOM: somatic complaint AUC: 0.9814945692932371
SYMPTOM: suicidal thoughts and attempts AUC: 0.9894754965004331
SYMPTOM: worthlessness AUC: 0.8942564931585546


In [98]:
roberta_scores_df = pd.DataFrame.from_dict(roberta_scores, orient='index', columns=['Score'])
roberta_scores_df.reset_index(inplace=True)
roberta_scores_df.columns = ['Symptom Type', 'Score']
print("roberta dataframe:\n", roberta_scores_df.dropna())

roberta dataframe:
                       Symptom Type     Score
0                            anger  0.985262
1                        anhedonia  0.987386
2                          anxiety  0.985909
4                disordered eating  0.988174
6                       loneliness  0.973580
7                         sad mood  0.979845
8                    self-loathing  0.978870
9                    sleep problem  0.991275
10               somatic complaint  0.981495
11  suicidal thoughts and attempts  0.989475
12                   worthlessness  0.894256
